# RCM Project: Coordinate Metadata Simplified
The coordinate system of the RCM project is difficult to understand, this notebook simplifies this by making all coordinates relative to the macroscopic image space.

In [76]:
# Imports
from dsa_helpers.girder_utils import login
from girder_client import GirderClient
import numpy as np
import ipywidgets as widgets
import pickle
import matplotlib.pyplot as plt
import cv2 as cv
from IPython.display import display
from tqdm.notebook import tqdm

In [43]:
# Functions
def convert_str_coords_to_float(coords: str) -> list[float]:
    """Convert a string of coordinates of form "[####, ####]" to a list of floats.
    
    Args:
        coords (str): The string of coordinates to convert.
        
    Returns:
        list[float]: The coordinates as floats.
        
    """
    return [float(coord) for coord in coords.strip("[]").split(",")]


def calculate_shape_of_image(metadata: dict) -> dict:
    """Calculate the shape of an image given the coordinates and the width and height.
    
    Args:
        metadata (dict): The metadata of the image.
        
    Returns:
        dict: The shape of the image
        
    """    
    # Get the size of the image by the pixels in metadata.
    coords = np.array([
        convert_str_coords_to_float(metadata["a"]),
        convert_str_coords_to_float(metadata["b"]),
        convert_str_coords_to_float(metadata["c"]),
        convert_str_coords_to_float(metadata["d"])
    ])
    
    min_x = min(coords[:, 0])
    max_x = max(coords[:, 0])
    min_y = min(coords[:, 1])
    max_y = max(coords[:, 1])
    
    # Return the width and height.
    return {"min_x": min_x, "max_x": max_x, "min_y": min_y, "max_y": max_y, 
            "width": max_x - min_x, "height": max_y - min_y}


def add_simplified_coordinates(gc: GirderClient, project_fld: dict) -> list[dict]:
    """For a given project folder, simplify the coordinates of each image to be 
    relative to the macroscopic image.
    
    Args:
        gc (girder_client.GirderClient): The Girder client to use.
        project_fld (dict): The project folder to process.
        
    Returns:
        list[dict]: The simplified coordinates of each image.
        
    """
    # Look for the registration metadata.
    reg_metadata = project_fld.get('meta', {}).get('registrationData', [])
    
    # Separate the macroscopic image from list from all other images.
    macroscopic_metadata = None
    other_metadata = []
    
    for img_metadata in reg_metadata:
        img_type = img_metadata.get("type")
        
        if img_type == "macroscopic image":
            macroscopic_metadata = img_metadata
        elif img_type in ("confocal image", "vivastack"):
            other_metadata.append(img_metadata)
            
    # If we don't have a macroscopic image, we can't do anything.
    metadata_to_add = []
    
    if macroscopic_metadata is not None:
        # Get the large image metadata for the macroscopic image.
        macroscopic_large_image_metadata = gc.get(
            f"item/{macroscopic_metadata['yamlId']}/tiles"
        )
        
        # Get the image width and height.
        macroscopic_width = macroscopic_large_image_metadata["sizeX"]
        macroscopic_height = macroscopic_large_image_metadata["sizeY"]
        
        # Get the size of the macroscopic image by the pixels in metadata.
        macroscopic_shape = calculate_shape_of_image(macroscopic_metadata)
        
        # Calculate ratio to go from scan pixel value to actual pixel value.
        pixel_factor = macroscopic_width / macroscopic_shape["width"]
        
        # Loop through each other image. 
        for img_metadata in other_metadata:
            # Get the coordinates of this image.
            coords = calculate_shape_of_image(img_metadata)
            
            # For each x value, sutract min_x of macroscopic image and scale.
            x_min = int((coords['min_x'] - macroscopic_shape['min_x']) * pixel_factor)
            x_max = int((coords['max_x'] - macroscopic_shape['min_x']) * pixel_factor)
            
            # Repeat fo ry values.
            y_min = int((coords['min_y'] - macroscopic_shape['min_y']) * pixel_factor)
            y_max = int((coords['max_y'] - macroscopic_shape['min_y']) * pixel_factor)
            
            metadata_to_add.append({
                "x_min": x_min, "x_max": x_max, "y_min": y_min, "y_max": y_max,
                # add additiona info
                "description": img_metadata.get("description", ""),
                "fov_mm_x": img_metadata.get("fov_mm_x", ""),
                "fov_mm_y": img_metadata.get("fov_mm_y", ""),
                "type": img_metadata.get("type", ""),
                "slices": img_metadata.get("slices", ""),
                "itemid": img_metadata.get("itemid", ""),
                "timestamp": img_metadata.get("timestamp", ""),
                "z_um_bottom": img_metadata.get("z_um_bottom", ""),
                "z_um_top": img_metadata.get("z_um_top", ""),
                "yamlId": img_metadata.get("yamlId", "")
            })
            
            
    if len(metadata_to_add):
        # Add the metadata.
        _ = gc.addMetadataToFolder(project_fld["_id"], 
                                  {"registrationDataSimplified":metadata_to_add})
            
    return metadata_to_add

In [3]:
# Authenticate girder client.
gc = login("https://wsi-deid.pathology.emory.edu/api/v1")

In [47]:
# Folder id with image folders.
root_fld_id = "66104727ee87269983c87412"

In [45]:
# Get the folders for each project, or group of images.
project_flds = list(gc.listFolder(root_fld_id))

In [ ]:
# Run the function for each project folder.
for project_fld in tqdm(project_flds, desc="Adding Simplified Coordinates"):
    if project_fld['_id'] not in ('666b3623fa042d3a8432a579', '666b3afafa042d3a8432a672'):
        _ = add_simplified_coordinates(gc, project_fld)

In [ ]:
# Get the folders again.
project_flds = list(gc.listFolder(root_fld_id))

# Use interactive view to see each image on the macroscopic image.
sample_project = project_flds[0]

# Get the macroscopic image.
reg_metadata = sample_project["meta"]["registrationData"]

macroscopic_img = None

for img_metadata in reg_metadata:
    if img_metadata.get("type") == "macroscopic image":
        # Get the image.
        macroscopic_img = gc.get(
            f"item/{img_metadata['yamlId']}/tiles/region?units=base_pixels&exact=false"
            "&encoding=pickle&jpegQuality=95&jpegSubsampling=0",
            jsonResp=False
        )
        
        # Convert from pickle to numpy.
        macroscopic_img = pickle.loads(macroscopic_img.content)
        break

# Get the simplified registration metadata.
reg_metadata = sample_project["meta"].get("registrationDataSimplified", [])

# To make this faster - preload the images.
for img_metadata in reg_metadata:
    img = gc.get(
        f"item/{img_metadata['yamlId']}/tiles/region?units=base_pixels&exact=false"
        "&encoding=pickle&jpegQuality=95&jpegSubsampling=0",
        jsonResp=False
    )
    
    # Convert from pickle to numpy.
    img_metadata["img"] = pickle.loads(img.content)[:, :, 0]


def show_image_on_macroscopic_image(idx: int):
    # Draw the box on the macroscopic image.
    img_metadata = reg_metadata[idx]
    
    # Get this image.
    small_img = img_metadata["img"]
    
    # Get the coordinates.
    x_min = img_metadata["x_min"]
    x_max = img_metadata["x_max"]
    y_min = img_metadata["y_min"]
    y_max = img_metadata["y_max"]
    
    
    # Draw the box.
    img = cv.rectangle(macroscopic_img.copy(), (x_min, y_min), (x_max, y_max), 
                       (0, 255, 0), 2)
    
    # Plot both images side by side.
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.imshow(small_img, cmap="gray")
    plt.show()
    
    
# Create the interactive view.
img_slider = widgets.IntSlider(value=0, min=0, max=len(reg_metadata) - 1, continuous_update=False)
out = widgets.interactive(show_image_on_macroscopic_image, idx=img_slider)

# Create the layout.
ui = widgets.VBox([img_slider, out.children[-1]])
display(ui)